In [ ]:
import sys
import csv
import spotipy
import spotipy.util as util
import os.path
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from os import listdir
from os.path import isfile, join

scope = 'user-library-read'
# You can get credentials from https://developer.spotify.com/dashboard/applications
username = 'vmal.chaudhari@gmail.com'
file_count = -1
path = '/home/vimal/Downloads/data/tracks/'
playlists_ = set([f for f in listdir(path) if isfile(join(path, f))])

def get_tracks_from_playlist(sp, playlist_id):
    if sp is None:
        sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    try:
        results = sp.playlist_tracks(playlist_id, fields='items(track(id,name,track_number,popularity)),next')
    except spotipy.client.SpotifyException:
        sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
        results = sp.playlist_tracks(playlist_id, fields='items(track(id,name,track_number,popularity)),next')
    tracks = results['items']
    tlist = []
    print(playlist_id," ",tracks)
    for track in tracks:
        #print(track['track']['id']," ",track['track']['track_number']," ",track['track']['popularity'])
        if track['track'] is None:
            return
        if track['track']['id']:
            tn = track['track']['track_number'] if track['track']['track_number'] else 0
            popu = track['track']['popularity'] if track['track']['popularity'] else 0
            tlist.append((track['track']['id'],tn,popu))
    if len(tlist) > 0:
        df = pd.DataFrame(tlist,columns=['id', 'track_number', 'popularity'])
        df.sort_values(by=['popularity'], inplace=True,ascending=False)
        df.to_csv(f"/home/vimal/Downloads/data/tracks/{playlist_id}.csv",index=False)
        print(playlist_id," is written.")
    else:
        print(playlist_id," has not tracks.")

def get_playlists():
    sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
    for search in range(ord('A'),ord('z')): #A to Z
        try:
            try:
                playlists = sp.search(chr(search), 50, 0, 'playlist')
            except spotipy.client.SpotifyException:
                sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
                playlists = sp.search(chr(search), 50, 0, 'playlist')
            for playlist in playlists['playlists']['items']:
                pid = playlist['id']
                if f"{pid}.csv" not in playlists_:
                    playlists_.add(f"{pid}.csv")
                    get_tracks_from_playlist(sp,pid)
            if len(playlists_) >= 10000:
                break
        except:
            print("failed")
                
#        while playlists:
#             with open('/home/vimal/Downloads/data/playlists.csv','a+',encoding='utf-8') as f:
#                 for playlist in playlists['playlists']['items']:
#                     #print(playlist['id'],playlist['name'])
#                     line = playlist['id'] + "\n"
#                     f.write(line)
        
# def get_tracks_from_playlists():
#     i = 0
#     sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
#     f = open('/home/vimal/Downloads/data/playlists.csv','r',encoding='utf-8')
#     while True:
#         line = f.readline()
#         if not line:
#             break
#         get_tracks_from_playlist(sp, line.rstrip('\n'))
#     f.close()    
# main
get_playlists()